# Machine Learning - Task E

**Part of IAFIG-RMS *Python for Bioimage Analysis* Course.**

*M Kundegorski*

2019-12-13

In this task you will use image information to cluster images into classes based on similarity of colour. In reality we could use any feature but colour is very visual and easy to follow.

There are 6 'classes' (colours) of blob, each of them has different value of hue (but the same mean intensity).

This task is based on a scikit-learn tutorial of k-means clustering: https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_iris.html#sphx-glr-auto-examples-cluster-plot-cluster-iris-py.

## The Task

In one experiment we were able to take a 'branbow'-esque dataset, i.e. every cell is expressing a combination of different fluorophores (colours), and each colour combination represents a type of cell. We imaged these cells with an RGB camera. These cells are sparse so we were easily able to segment them, find their bounding box and create a database of images each containing a single cell. We want to use clustering to automatically identify which cell belongs to which cell-type. Luckily, we've convinced a PhD student to manually label our sample database so we can confirm our results at the end.

Given the colour of our cells we want to automatically, and in an unsupervised fashion, 'cluster' our cells into classes, ignoring other features like shape or size.

1. Let's start by loading our data and visualising the manual class labels in RGB space (other colour spaces would also work)
2. Use k-means clustering to cluster our data and compare it to our manual labels
3. Advanced: Use sum of square errors to identify the correct 'k' to use
4. Advanced: Use mean shift clustering to cluster our data and compare it to our manual labels

## Task E.1

Let's start by loading data. Later on, you can change parameters to see how the clustering results change with different numbers and noise.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

# Utils is a custom module written to simplify these tutorials
# You do not need to understand these codes for this practical
from utils.practice_data import generateBlobsData  # this loads data into a DataFrame
from utils.practice_data import showBlobs  # this allows quick visualisation of the data

imageDir = './assets/simple_blobs/'
number_of_classes = 6
# Is 30 a sufficent number of samples? How much noise can our algorithms deal with?
number_of_samples = 30
noise = 10 #value from 0 to 250
problem = generateBlobsData(imageDir, number_of_classes, number_of_samples, imSize = 100, colour=True, noiseSize = noise)

# Visualise
display(problem.loc[:,'class'].describe())  # describe classes
showBlobs(problem.sample(12))

## Task E.2

Time for some data wrangling - as usual the data needs to have right datatype required by the algorithm. For this simple clustering example we're going to extract the mean 'red', the mean 'green' and the mean 'blue' value for each cell. To make things simply, given we've only got one cell per image, we will use the means for the whole image. How might this affect our means and therefore our clustering? Should we use the mean or the median?

Run the following cell to get the median colour channel values.

In [ ]:
y=problem.loc[:,'class'].values.astype(int)  # Convert classes to int

for cell in problem.index:
    problem.loc[cell,'mean-R'] = problem.loc[cell,'raw_data'][:,:,0].mean()  # mean of the first (red) channel of our image data for this cell
    problem.loc[cell,'mean-G'] = problem.loc[cell,'raw_data'][:,:,1].mean()  # mean of the second (green) channel of our image data for this cell
    problem.loc[cell,'mean-B'] = problem.loc[cell,'raw_data'][:,:,2].mean()  # mean of the third (blue) channel of our image data for this cell

problem.describe()

Run the following cell to plot our cells on three axes - one for R, one for G and one for B. This code also colourcodes cells by their class (the manual class at the moment).

In [ ]:
f, axis = plt.subplots(1,1,subplot_kw={'projection':'3d'})  # create a figure with a single 3d axis (subplot)

# Get a list of unique classes
blob_classes = problem.loc[:,'class'].unique()

# Plot each blob and label
for blob_class in blob_classes:
    axis.plot(problem[problem['class']==blob_class]['mean-R'],
            problem[problem['class']==blob_class]['mean-G'],
            problem[problem['class']==blob_class]['mean-B'], 
            'o', label=blob_class)
axis.set_ylabel('Red')
axis.set_xlabel('Green')
axis.set_zlabel('Blue')

axis.legend(loc='upper left', numpoints=1, ncol=3, fontsize=8, bbox_to_anchor=(0, 0))  # add a legend

plt.show()

## Task E.3

Now you will use [sklearn.cluster.kMeans()](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) to perform k-means clustering on your data with k=6. This clustering algorithm never sees your manual class labels. Let's see if we can put your PhD student out of a job - I'm sure you have plenty more for them to do!

Run the next cell to do this.

In [ ]:
from sklearn import cluster

# Initialise clustering algorithm
k_means = cluster.KMeans(n_clusters=6)  # How would you decide how many clusters to choose?

# Perform fit with colour data only (no class labels)
k_means.fit(problem[['mean-R','mean-G','mean-B']])

# Extract cluster labels
problem['kMeans-cluster'] = k_means.labels_

Run the next cell to plot your PhD student's clustering and the k-means clustering side-by-side. Note that the cluster *numbers* may not map exactly but the cluster members should be similar.

In [ ]:
f, ax = plt.subplots(1,2,subplot_kw={'projection':'3d'})  # create a figure with two 3d axes (subplot)
(axPhD, axKM) = ax.flatten()  # give each subplot a unique name

# Get a list of unique PhD-student-derived classes
phd_classes = problem.loc[:,'class'].unique()

# Plot each blob and label
for blob_class in blob_classes:
    axPhD.plot(problem[problem['class']==blob_class]['mean-R'],
            problem[problem['class']==blob_class]['mean-G'],
            problem[problem['class']==blob_class]['mean-B'], 
            'o', label=blob_class)
axPhD.set_ylabel('Red')
axPhD.set_xlabel('Green')
axPhD.set_zlabel('Blue')

axPhD.legend(loc='upper left', numpoints=1, ncol=3, fontsize=8, bbox_to_anchor=(0, 0))  # add a legend

# Get a list of unique k-Means-derived classes
km_classes = problem.loc[:,'kMeans-cluster'].unique()

# Plot each blob and label
for blob_class in blob_classes:
    axKM.plot(problem[problem['class']==blob_class]['mean-R'],
            problem[problem['class']==blob_class]['mean-G'],
            problem[problem['class']==blob_class]['mean-B'], 
            'o', label=blob_class)
axKM.set_ylabel('Red')
axKM.set_xlabel('Green')
axKM.set_zlabel('Blue')

axKM.legend(loc='upper left', numpoints=1, ncol=3, fontsize=8, bbox_to_anchor=(0, 0))  # add a legend

plt.show()

## Advanced Task E.4

So far, we used our senses to guess that we needed 6 clusters. However, there is a relatively simple way to estimate the number of clusters we want from a k-means algorithm. First, we guess a reasonable range, for example, 1 to 10 clusters, and run k-means clustering for each guess. We then look for a 'knee' point in Sum of Square Errors (SSE) for each clustering. The SSE (or intertia) is the sum of distances from each sample (each cell) to the centre of it's cluster, e.g. the mean position of all cells in that cluster.

You can access value of SSE using `.inertia_` attribute of a fitted k-means model.

Use a for loop, and by copying the code above, create a cell to first the best 'k' to use for this data.

In [ ]:
### Remove this cell ###
sse = np.zeros(9)  # create an array to hold our sse values

for k in np.arange(1, 10):
    # Initialise clustering algorithm
    k_means = cluster.KMeans(n_clusters=k)  # How would you decide how many clusters to choose?

    # Perform fit with colour data only (no class labels)
    k_means.fit(problem[['mean-R','mean-G','mean-B']])

    # Extract inertia
    sse[k] = kmeans.inertia_

# Plot SSE for different k
f, axis = plt.subplots(1,1)  # create a figure with a single axis (subplot)

axis.plot(np.arange(1,10), sse)
axis.set_ylabel('SSE')
axis.set_xlabel('Number of Clusters (k)')

plt.show()
### Remove this cell ###

Mean Shift Clustering: What happens if we change `bandwidth` parameter?

In [ ]:
from sklearn.cluster import MeanShift
ms = MeanShift(bandwidth=10).fit(problem[['ch1','ch2','ch3']])
problem['cluster_ms'] = ms.labels_

Let's compare kmeans and Mean Shift clustering with ground-truth data

In [ ]:
#show cluster
fig = plt.figure(figsize=plt.figaspect(0.3))
ax1 = fig.add_subplot(1, 3, 1, projection='3d')
ax2 = fig.add_subplot(1, 3, 2, projection='3d')
ax3 = fig.add_subplot(1, 3, 3, projection='3d')

blob_classes = problem['class'].unique()
for blob_class in blob_classes:
    ax1.plot(problem[problem['class']==blob_class]['ch1'],
            problem[problem['class']==blob_class]['ch2'],
            problem[problem['class']==blob_class]['ch3'], 
            'o', label=blob_class)
ax1.set_title('Ground truth label')    
blob_classes = problem['cluster'].unique()
for blob_class in blob_classes:
    ax2.plot(problem[problem['cluster']==blob_class]['ch1'],
            problem[problem['cluster']==blob_class]['ch2'],
            problem[problem['cluster']==blob_class]['ch3'], 
            'o', label=blob_class)
ax2.set_title('k-means with {} cluster'.format(len(blob_classes)))
    
blob_classes = problem['cluster_ms'].unique()
for blob_class in blob_classes:
    ax3.plot(problem[problem['cluster_ms']==blob_class]['ch1'],
            problem[problem['cluster_ms']==blob_class]['ch2'],
            problem[problem['cluster_ms']==blob_class]['ch3'], 
            'o', label=blob_class)
ax3.set_title('Mean shift clustering with {} clusters'.format(len(blob_classes)))    
    
ax1.legend(loc='upper left', numpoints=1, ncol=3, fontsize=8, bbox_to_anchor=(0, 0))
ax2.legend(loc='upper left', numpoints=1, ncol=3, fontsize=8, bbox_to_anchor=(0, 0))
ax3.legend(loc='upper left', numpoints=1, ncol=3, fontsize=8, bbox_to_anchor=(0, 0))

plt.show()